### Subset Distributions

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from wakeful import log_munger, metrics, virus_total
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from bat.dataframe_to_matrix import DataFrameToMatrix
%matplotlib inline

#### dnscat2

In [37]:
df_train_conn_dnscat2 = log_munger.hdf5_to_df('dnscat2-2017-12-31-conn-train', './data')
df_test_conn_dnscat2 = log_munger.hdf5_to_df('dnscat2-2017-12-31-conn-test', './data')
print('train value counts:\n', df_train_conn_dnscat2.label.value_counts())
print('test value counts:\n', df_test_conn_dnscat2.label.value_counts())

train value counts:
 0    27255
1    25770
Name: label, dtype: int64
test value counts:
 0    27226
1     1515
Name: label, dtype: int64


In [38]:
df_train_conn_dnscat2 = df_train_conn_dnscat2.dropna(axis=0, how='any')

In [39]:
y_df_train_conn_dnscat2 = df_train_conn_dnscat2.pop('label')
to_matrix = DataFrameToMatrix()
X_matrix_train_conn_dnscat2 = to_matrix.fit_transform(df_train_conn_dnscat2)


Changing column conn_state to category...
Changing column proto to category...
Changing column service to category...
Changing column tunnel_parents to category...
Normalizing column id.orig_p...
Normalizing column id.resp_p...
Normalizing column missed_bytes...
Normalizing column orig_bytes...
Normalizing column orig_ip_bytes...
Normalizing column orig_pkts...
Normalizing column resp_bytes...
Normalizing column resp_ip_bytes...
Normalizing column resp_pkts...
Normalizing column pcr...


### PCA Feature Reduction

In [40]:
pca = PCA(n_components=6)
pca.fit(X_matrix_train_conn_dnscat2)
print(sum(pca.explained_variance_ratio_), '=', pca.explained_variance_ratio_)
print(pca.n_components_)
X_matrix_train_conn_dnscat2 = pca.fit_transform(X_matrix_train_conn_dnscat2)

0.910998254716 = [ 0.39508562  0.21017696  0.16128267  0.06264435  0.04276256  0.0390461 ]
6


In [35]:
X_matrix_train_conn_dnscat2.shape

(46183, 6)

### SequentialFeature Reduction

In [ ]:
sfs1 = SFS(knn, 
           k_features=3, 
           forward=True, 
           floating=False, 
           verbose=2,
           scoring='accuracy',
           cv=0)

sfs1 = sfs1.fit(X, y)

In [ ]:
sns.pairplot(df_train_conn_dnscat2, hue='label')

### Clustering

In [41]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn import metrics
from wakeful import log_munger
import os
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
%pwd

'/Users/rob/Google_Drive/Datascience/Galvanize/Project/wakeful_proj'